In [1]:
from models.energy import energy_quantile_regression
from models.energy import energy_regression_temp

In [2]:
from functions import get_energy

In [3]:
energydata = get_energy.get()

  0%|          | 0/263 [00:00<?, ?it/s]/Users/stephantimpe/PycharmProjects/probabilistic-forecasting-challenge/functions/get_energy.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 263/263 [00:28<00:00,  9.28it/s]


In [4]:
#import read csv
from pandas import read_csv

In [5]:
temp_data = read_csv('temperatures.csv')

In [6]:
temp_data=temp_data.drop(columns=['Unnamed: 0'])

In [7]:
#set date as index
temp_data['date'] = temp_data['date'].astype('datetime64[ns]')

In [8]:
temp_data.index = temp_data['date']

In [9]:
temp_data=temp_data.drop(columns=['date'])

In [10]:
import pandas as pd

In [11]:
#merhe energy and temperature data on index
energydata.index = pd.to_datetime(energydata.index)
temp_data.index = pd.to_datetime(temp_data.index)
energydata = energydata.merge(temp_data, left_index=True, right_index=True)

In [12]:
energydata

,gesamt,weekday,temperature_2m
2018-12-24 00:00:00,42.02925,0,4.815
2018-12-24 01:00:00,39.61025,0,4.665
2018-12-24 02:00:00,39.13875,0,3.915
2018-12-24 03:00:00,39.42100,0,3.365
2018-12-24 04:00:00,40.74775,0,2.965
...,...,...,...
2023-12-11 19:00:00,65.03775,0,7.165
2023-12-11 20:00:00,61.46575,0,7.115
2023-12-11 21:00:00,57.69975,0,7.115
2023-12-11 22:00:00,54.27875,0,6.715


In [13]:
reg= energy_regression_temp.energy_quantile_regression_temp(energydata,temperature_forecast=[8,9,6,4,3,3],date_str='2023-12-20')

                         QuantReg Regression Results                          
Dep. Variable:                 gesamt   Pseudo R-squared:               0.5082
Model:                       QuantReg   Bandwidth:                      0.6984
Method:                 Least Squares   Sparsity:                        11.93
Date:                Thu, 04 Jan 2024   No. Observations:                43536
Time:                        22:27:00   Df Residuals:                    43499
                                        Df Model:                           36
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             58.7713      0.174    338.165      0.000      58.431      59.112
weekday           -1.8375      0.014   -128.626      0.000      -1.865      -1.809
temperature_2m    -0.1226      0.007    -16.457      0.000      -0.137      -0.108
month_2            0.6158      0

In [14]:
reg

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-12-20,energy,36 hour,39.281182,47.789784,52.277807,55.28527,57.96702
1,2023-12-20,energy,40 hour,51.067257,61.134479,65.266133,69.948076,74.691423
2,2023-12-20,energy,44 hour,49.136816,57.356292,60.579088,63.718057,67.169442
3,2023-12-20,energy,60 hour,39.198105,46.567444,50.930858,54.228381,57.454846
4,2023-12-20,energy,64 hour,51.437144,60.237694,64.164448,69.074701,74.311288
5,2023-12-20,energy,68 hour,48.827256,55.971174,59.109507,62.56941,66.59125


In [35]:
reg

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-12-20,energy,36 hour,39.281182,47.789784,52.277807,55.28527,57.96702
1,2023-12-20,energy,40 hour,51.067257,61.134479,65.266133,69.948076,74.691423
2,2023-12-20,energy,44 hour,49.136816,57.356292,60.579088,63.718057,67.169442
3,2023-12-20,energy,60 hour,39.198105,46.567444,50.930858,54.228381,57.454846
4,2023-12-20,energy,64 hour,51.437144,60.237694,64.164448,69.074701,74.311288
5,2023-12-20,energy,68 hour,48.827256,55.971174,59.109507,62.56941,66.59125


In [36]:
from models.DAX import DAX_quantile_regression, DAX_baseline
from models.energy import energy_quantile_regression
from functions import get_energy, get_DAX, merge_submissions, check_submission
from functions.prepare_data import split_time
from functions.evaluation import evaluate_horizon
from tqdm import tqdm
from functions import evaluate_dax
from functions import naive_ensemble

daxdata = get_DAX.get()
base = DAX_baseline.DAX_baseline(daxdata)
base
quantile = DAX_quantile_regression.DAX_quantile_regression(daxdata)
df_dax = naive_ensemble.ensemble(base, quantile, 0.5, 0.5)

In [37]:
from models.DAX import DAX_quantile_regression, DAX_baseline
from models.energy import energy_quantile_regression
from functions import get_energy,get_DAX,merge_submissions,check_submission
from functions.prepare_data import split_time
from functions.evaluation import evaluate_horizon
from tqdm import tqdm
from functions import evaluate_model
from models.energy import energy_baseline

In [38]:
df_sub = merge_submissions.merge(reg, df_dax)

  forecast_date      target  horizon     q0.025      q0.25       q0.5  \
0    2023-12-20         DAX    1 day  -2.889883  -0.553345   0.063023   
1    2023-12-20         DAX    2 day  -2.823391  -0.241397    0.58968   
2    2023-12-20         DAX    5 day  -4.088016  -0.930304    0.00483   
3    2023-12-20         DAX    6 day  -4.645894  -1.060094  -0.012574   
4    2023-12-20         DAX    7 day  -4.840274  -1.196746  -0.074314   
0    2023-12-20      energy  36 hour  39.281182  47.789784  52.277807   
1    2023-12-20      energy  40 hour  51.067257  61.134479  65.266133   
2    2023-12-20      energy  44 hour  49.136816  57.356292  60.579088   
3    2023-12-20      energy  60 hour  39.198105  46.567444  50.930858   
4    2023-12-20      energy  64 hour  51.437144  60.237694  64.164448   
5    2023-12-20      energy  68 hour  48.827256  55.971174  59.109507   
0    2023-12-20  infections   0 week         NA         NA         NA   
1    2023-12-20  infections   1 week         NA    

In [39]:
df_sub

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-12-20,DAX,1 day,-2.889883,-0.553345,0.063023,0.678244,2.289265
1,2023-12-20,DAX,2 day,-2.823391,-0.241397,0.58968,1.336447,3.44167
2,2023-12-20,DAX,5 day,-4.088016,-0.930304,0.00483,0.863911,3.226479
3,2023-12-20,DAX,6 day,-4.645894,-1.060094,-0.012574,0.94207,3.582623
4,2023-12-20,DAX,7 day,-4.840274,-1.196746,-0.074314,0.861862,3.964774
0,2023-12-20,energy,36 hour,39.281182,47.789784,52.277807,55.28527,57.96702
1,2023-12-20,energy,40 hour,51.067257,61.134479,65.266133,69.948076,74.691423
2,2023-12-20,energy,44 hour,49.136816,57.356292,60.579088,63.718057,67.169442
3,2023-12-20,energy,60 hour,39.198105,46.567444,50.930858,54.228381,57.454846
4,2023-12-20,energy,64 hour,51.437144,60.237694,64.164448,69.074701,74.311288


In [40]:
df_sub.to_csv('../../forecasts/20231220_JonSnow.csv',index=False)